In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # Import tqdm for the progress bar


2023-08-18 20:26:42.450370: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-18 20:26:43.775108: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 20:26:45.845860: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load the CSV data
data = pd.read_csv('/home/woody/iwso/iwso092h/student_summaries/commonlit-evaluate-student-summaries/summaries_train.csv')

# Preprocess the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_seq_length = 512

def tokenize_text(text):
    return tokenizer.encode(text, add_special_tokens=True, max_length=max_seq_length, padding='max_length', truncation=True)

data['text_tokens'] = data['text'].apply(tokenize_text)

# Split the data into features and targets
X = data['text_tokens'].tolist()
y = data[['wording', 'content']].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [12]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Generate BERT embeddings
def generate_bert_embeddings(text_tokens):
    with tqdm(total=len(text_tokens)) as pbar:
        with torch.no_grad():
            outputs = bert_model(text_tokens)
            embeddings = outputs.last_hidden_state[:, 0, :]  # Extract embeddings for [CLS] token
            pbar.update(1)
        return embeddings

X_train_embeddings = generate_bert_embeddings(X_train)
X_test_embeddings = generate_bert_embeddings(X_test)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


: 

: 